In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import pandas as pd 

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


/tmp/ipykernel_1771/2851567373.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
# df = 
# shape = df.shape
# print(shape)
# print(df.head())

# training, test = torch.utils.data.random_split(df, [int(shape[0]*0.8), int(shape[0]*0.2)])
training = pd.read_csv("data/ParisHousingClass.csv", sep=",", header="infer")
test = pd.read_csv("data/ParisHousingClass_Test.csv", sep=",", header="infer")
features = training.shape[1] - 1

print(f"Training set size: {len(training)}")
print(f"Eval set size: {len(test)}")

categories = {
    "Basic": 0, "Luxury": 1
}

Training set size: 8000
Eval set size: 2000


In [6]:
class ParisCategoryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(features, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [10]:
model = ParisCategoryClassifier().to(device)
print(model)

class ParisDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def __len__(self):
        print(self.df.shape)
        return self.df.size

    def __getitem__(self, idx):
        data = self.df.drop(columns=['category'])
        label = self.df['category']

        return data, label

training_set = ParisDataset(training)
test_set = ParisDataset(test)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=False)

print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(test_set)))



ParisCategoryClassifier(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


AttributeError: 'Subset' object has no attribute 'shape'